In [19]:
import numpy as np 
import cv2 
from matplotlib import pyplot as plt

# 读入图像
img1 = cv2.imread("assets/box.png", 0)
img2 = cv2.imread("assets/box_in_scene.png", 0)

# 创建SIFT
sift = cv2.SIFT_create()

# 找关键点和计算特征
kp1, des1 = sift.detectAndCompute(img1, None)
kp2, des2 = sift.detectAndCompute(img2, None)

# 寻找匹配的关键点
FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
seach_params = dict(checks = 50)

good = []
flann = cv2.FlannBasedMatcher(index_params, seach_params)
matches = flann.knnMatch(des1, des2, k=2)
for m, n in matches:
    if m.distance < 0.7*n.distance:
        good.append(m)


# 找到映射关系
MIN_MATCH_COUNT = 10
pts__ = None 
dst__ = None 
if len(good) > MIN_MATCH_COUNT:
    src_pts = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    matchesMask = mask.ravel().tolist()
    h, w = img1.shape
    pts__ = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1, 1, 2)
    dst__ = cv2.perspectiveTransform(pts__, M)
    # img2 = cv2.polylines(img2, [np.int32(dst__)], True, 255, 3, cv2.LINE_AA)
else:
    print("Not enough matches are found - {}/{}".format(len(good), MIN_MATCH_COUNT))
    matchesMask = None

# 绘制匹配结果
draw_params = dict(matchColor=(0, 255, 0),  # draw matches in green color
                singlePointColor=None,
                matchesMask=matchesMask,  # draw only inliers
                flags=2)
img3 = cv2.drawMatches(img1, kp1, img2, kp2, good, None, **draw_params)

# 对应的点发生了偏移
shift_x = np.float32([[w-1, 0], [w-1, 0], [w-1, 0], [w-1, 0]]).reshape(-1, 1, 2)
dst2__ = np.int32(dst__ + shift_x)

# 绘制原始图像图像的位置和查找目标的位置
img3 = cv2.polylines(img3, [np.int32(pts__)], True, 
                    (0, 0, 255), 2, cv2.LINE_AA)
img3 = cv2.polylines(img3, [np.int32(dst2__)], True,
                    (0, 0, 255), 2, cv2.LINE_AA)

cv2.imshow("result", img3)
cv2.waitKey(0)
cv2.destroyAllWindows()
